# Defog Quick Start Demo (15 minutes)

Welcome to the defog-python library! This notebook demonstrates the core capabilities:
- Chat with multiple LLM providers using a unified interface
- Extract structured data from PDFs and images
- Query databases with natural language
- Use agents for complex reasoning tasks

**Time:** ~15 minutes | **Level:** Beginner | **Prerequisites:** API keys for at least one provider

## 1. Setup and Installation

First, let's install defog-python and set up our environment.

In [45]:
# Install defog-python (uncomment if not already installed)
# !pip install defog-python[all]

# Import required libraries
import os
import pandas as pd
from typing import List
from pydantic import BaseModel, Field
from IPython.display import display, Markdown, HTML, Image
import warnings

warnings.filterwarnings("ignore")

# Import defog components
from defog.llm.utils import chat_async
from defog.llm.pdf_data_extractor import PDFDataExtractor
from defog.llm.image_data_extractor import ImageDataExtractor
from defog.llm.sql import sql_answer_tool
import duckdb

## 2. Configure API Keys

Set up your API keys. You only need one provider to get started!

In [46]:
# Option 1: Set environment variables (recommended)
# os.environ["OPENAI_API_KEY"] = "your-openai-key"
# os.environ["ANTHROPIC_API_KEY"] = "your-anthropic-key"
# os.environ["GEMINI_API_KEY"] = "your-gemini-key"

# Option 2: Load from .env file
# from dotenv import load_dotenv
# load_dotenv()

# Check which providers are available
available_providers = []
for provider in ["OPENAI", "ANTHROPIC", "GEMINI"]:
    if os.getenv(f"{provider}_API_KEY"):
        available_providers.append(provider.lower())

print(f"Available providers: {available_providers}")
if not available_providers:
    print("⚠️  No API keys found! Please set at least one provider's API key above.")
else:
    default_provider = available_providers[0]
    default_api_key = os.getenv(f"{default_provider.upper()}_API_KEY")

Available providers: ['openai', 'anthropic', 'gemini']


## 3. Simple Chat - Unified LLM Interface

The `chat_async` function provides a unified interface for all LLM providers.

In [ ]:
# Simple chat example
messages = [
    {
        "role": "user",
        "content": "What are the main benefits of using LLMs for data analysis?",
    }
]

response = await chat_async(
    messages=messages,
    provider="openai",
    model="gpt-4.1",
    temperature=0.7,
    max_completion_tokens=1000,
)

display(Markdown(f"**Response:** {response.content}"))

**Response:** Large Language Models (LLMs) like GPT-4 offer several compelling benefits for data analysis:

1. **Natural Language Interaction**  
   - **Query Data in Plain English:** LLMs let users ask questions about data using natural language, reducing the need to write complex queries or code.
   - **Lower Barrier to Entry:** Non-technical users can analyze data without deep knowledge of programming or statistics.

2. **Automated Data Processing**  
   - **Data Cleaning & Transformation:** LLMs can help identify inconsistencies, suggest corrections, and automate cleaning steps.
   - **Data Summarization:** They quickly summarize large, complex datasets or reports, highlighting trends and anomalies.

3. **Insight Generation**  
   - **Pattern Recognition:** LLMs can surface non-obvious patterns or correlations in data.
   - **Hypothesis Generation:** They can suggest potential hypotheses or follow-up analyses based on data context.

4. **Code Generation & Assistance**  
   - **Automate Analysis Scripts:** LLMs generate Python, R, SQL, or other code snippets for tasks like visualization, statistical tests, or machine learning.
   - **Debugging Help:** They assist in troubleshooting code or suggesting improvements.

5. **Visualization Support**  
   - **Chart Recommendations:** LLMs can propose appropriate charts/graphs for various data types and generate code to create them.
   - **Interpretation of Visuals:** They help explain what a chart or graph means in plain language.

6. **Documentation & Reporting**  
   - **Auto-generate Reports:** LLMs can draft executive summaries, reports, or presentations based on analysis results.
   - **Explain Methods:** They document analytical steps, making results more transparent and reproducible.

7. **Scalability & Accessibility**  
   - **Handle Large Volumes:** LLMs can help process and interpret vast datasets efficiently.
   - **Integration:** They can be integrated into business intelligence tools, chatbots, and dashboards for real-time assistance.

**Limitations to Remember:**  
- LLMs may hallucinate or misinterpret data if not provided with clear, structured input.
- They often require coupling with dedicated data processing tools for handling raw or very large datasets.

**Summary:**  
LLMs democratize data analysis by improving accessibility, automating routine tasks, and speeding up the process of insight generation, making data-driven decision-making more efficient and inclusive.

## 4. Structured Output with Pydantic

Get structured, validated responses using Pydantic models.

In [49]:
# Define a structured output format
class CricketInsight(BaseModel):
    topic: str = Field(description="Aspect of cricket being analyzed")
    insight: str = Field(description="Key insight or finding")
    relevance: str = Field(description="Why this matters")


class CricketAnalysis(BaseModel):
    insights: List[CricketInsight] = Field(description="List of cricket insights")


# Get structured response
messages = [
    {
        "role": "user",
        "content": "What are 3 key factors that contribute to winning matches in limited overs cricket?",
    }
]

analysis = await chat_async(
    messages=messages,
    provider="gemini",
    model="gemini-2.5-flash",
    response_format=CricketAnalysis,
    temperature=0.7,
)

# Display as a nice table
df = pd.DataFrame([insight.model_dump() for insight in analysis.content.insights])
display(HTML(df.to_html(index=False)))

topic,insight,relevance
Explosive Batting & Depth,"Teams with strong batting depth that can accelerate scoring in the middle and death overs, along with a solid opening partnership, consistently post winning totals or chase targets effectively.","High scores put immense pressure on the opposition, while effective chases demonstrate composure and power-hitting, both vital for limited-overs success."
Wicket-Taking Bowling & Variations,"The ability of bowlers to take wickets at regular intervals, particularly through effective variations (pace, spin, slower balls), prevents opposition partnerships from flourishing and restricts run flow.","Wickets are the primary way to control the game and limit the opposition's score, crucial for both setting and chasing targets."
Exceptional Fielding & Catching,"Elite fielding units that save crucial runs, execute run-outs, and hold onto difficult catches significantly impact match outcomes by creating pressure and removing key batsmen.","Every run saved and every wicket taken through fielding directly contributes to the team's total or limits the opponent's, often turning close matches in their favor."


## 5. Natural Language Database Queries - Cricket World Cup 2015

Query the Cricket World Cup database using natural language!

In [50]:
# Connect to the cricket database
db_conn = duckdb.connect("../sample_data/cricket_wc2015.duckdb")

# See what tables we have
tables = db_conn.execute("SHOW TABLES").fetchall()
print(f"Available tables: {[t[0] for t in tables]}")

Available tables: ['ball_by_ball', 'match_summary', 'player_batting_stats', 'player_bowling_stats', 'team_performance']


In [17]:
# Query 1: Top run scorers
result = await sql_answer_tool(
    question="Who were the top 5 run scorers in the tournament?",
    db_type="duckdb",
    db_creds={
        "database": "/Users/rishabh/defog/defog-python/sample_data/cricket_wc2015.duckdb"
    },
    provider="anthropic",
    model="claude-sonnet-4-20250514",
)

if result["success"]:
    print(f"Generated SQL: {result['query']}\n")
    df = pd.DataFrame(result["results"], columns=result["columns"])
    display(df)
else:
    print(f"Error: {result['error']}")

─────────────────────────────────────────────── 🔧 SQL Query Answer ───────────────────────────────────────────────

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│                                                                                                                 │
│  Answering: Who were the top 5 run scorers in the tournament?                                                   │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Provider: anthropic | Model: claude-sonnet-4-20250514

⚡ Extracting table metadata

Getting schema for each table that you selected...

⚡ Converting question to SQL

⚡ Running query on database

Completed in 6.8s

Generated SQL: SELECT batsman_name, runs_scored
FROM player_batting_stats
ORDER BY runs_scored DESC
LIMIT 5;



,batsman_name,runs_scored
0,Martin Guptill,547.0
1,Kumar Sangakkara,541.0
2,AB de Villiers,482.0
3,Brendan Taylor,433.0
4,Shikhar Dhawan,412.0


## 6. PDF Data Extraction - Apple Financials

Extract structured data from PDF documents.

In [55]:
# Initialize PDF extractor
pdf_extractor = PDFDataExtractor(
    analysis_provider="anthropic",
    analysis_model="claude-opus-4-20250514",
    extraction_provider="anthropic",
    extraction_model="claude-opus-4-20250514",
    max_parallel_extractions=10,
)

# Extract data
display(Markdown("### Extracting data from Apple's financial report..."))

result = await pdf_extractor.extract_all_data(
    pdf_url="https://www.apple.com/newsroom/pdfs/fy2025-q2/FY25_Q2_Consolidated_Financial_Statements.pdf",
)

print(f"\nDocument Type: {result.document_type}")
print(f"Total datapoints identified: {result.total_datapoints_identified}")
print(f"Successful extractions: {result.successful_extractions}")
print(f"Failed extractions: {result.failed_extractions}")
print(f"Total time: {result.total_time_ms / 1000:.2f} seconds")

print("\n--- Cost Analysis ---")
print(f"Total cost: ${result.total_cost_cents / 100:.4f}")
print(
    f"Analysis cost (Step 1): ${result.metadata.get('analysis_cost_cents', 0.0) / 100:.4f}"
)
print(
    f"Extraction cost (Step 2+): ${result.metadata.get('extraction_cost_cents', 0.0) / 100:.4f}"
)

print("\n--- Token Usage ---")
print(f"Total input tokens: {result.metadata.get('total_input_tokens', 0):,}")
print(f"Total output tokens: {result.metadata.get('total_output_tokens', 0):,}")
print(f"Total cached tokens: {result.metadata.get('total_cached_tokens', 0):,}")
print(
    f"Total tokens: {result.metadata.get('total_input_tokens', 0) + result.metadata.get('total_output_tokens', 0):,}"
)

print("\n--- Extracted Datapoints ---")
for extraction in result.extraction_results:
    if extraction.success:
        print(f"\n✅ {extraction.datapoint_name}:")
        print(
            f"   Cost: ${extraction.cost_cents / 100:.4f} | Tokens: {extraction.input_tokens + extraction.output_tokens:,} (in:{extraction.input_tokens:,}, out:{extraction.output_tokens:,}, cached:{extraction.cached_tokens:,})"
        )
    else:
        print(f"\n❌ {extraction.datapoint_name}: {extraction.error}")
        if extraction.cost_cents > 0:
            print(
                f"   Cost: ${extraction.cost_cents / 100:.4f} | Tokens: {extraction.input_tokens + extraction.output_tokens:,}"
            )

### Extracting data from Apple's financial report...


Document Type: unknown
Total datapoints identified: 0
Successful extractions: 0
Failed extractions: 0
Total time: 6.34 seconds

--- Cost Analysis ---
Total cost: $0.0274
Analysis cost (Step 1): $0.0274
Extraction cost (Step 2+): $0.0000

--- Token Usage ---
Total input tokens: 1,174
Total output tokens: 130
Total cached tokens: 0
Total tokens: 1,304

--- Extracted Datapoints ---


## 7. Image Data Extraction - Carbon Budget Visualization

Extract data from charts and visualizations in images.

In [58]:
# Display the image
display(Markdown("### Carbon Budget Visualization"))
display(
    Image(
        url="https://ourworldindata.org/cdn-cgi/imagedelivery/qLq-8BTgXU8yG0N6HnOy8g/1cea1569-d611-4d7f-7a9c-d01f81fab400/w=5671",
        width=600,
    )
)

### Carbon Budget Visualization

In [57]:
# Initialize image extractor
image_extractor = ImageDataExtractor(
    analysis_provider="openai",
    analysis_model="o3",
    extraction_provider="openai",
    extraction_model="o3",
)

# Extract data from image
carbon_data = await image_extractor.extract_all_data(
    image_url="https://ourworldindata.org/cdn-cgi/imagedelivery/qLq-8BTgXU8yG0N6HnOy8g/1cea1569-d611-4d7f-7a9c-d01f81fab400/w=5671",
)

# Display extracted data
print(carbon_data)

image_url='https://ourworldindata.org/cdn-cgi/imagedelivery/qLq-8BTgXU8yG0N6HnOy8g/1cea1569-d611-4d7f-7a9c-d01f81fab400/w=5671' image_type='infographic' total_datapoints_identified=2 successful_extractions=2 failed_extractions=0 extraction_results=[DataExtractionResult(datapoint_name='carbon_budget_for_1_5c', success=True, extracted_data=carbon_budget_for_1_5c(columns=['category', 'co2_billion_tonnes'], data=[['17%', 500], ['33%', 300], ['50%', 250], ['67%', 150], ['83%', 100], ['Global emissions 2022', 41]], chart_type='bar', row_count=6), error=None, cost_cents=2.797, input_tokens=1421, output_tokens=344, cached_tokens=0), DataExtractionResult(datapoint_name='carbon_budget_for_2c', success=True, extracted_data=carbon_budget_for_2c(columns=['category', 'co2_billion_tonnes'], data=[['17%', 2000], ['33%', 1450], ['50%', 1150], ['67%', 950], ['83%', 800], ['Global emissions 2022', 41]], chart_type='bar', row_count=6), error=None, cost_cents=2.791, input_tokens=1411, output_tokens=345, ca

## Agent Orchestrator

In [ ]:
# port over example from examples/orchestrator_dynamic_example.py



## Summary & Next Steps

Congratulations! You've explored the core capabilities of defog-python:

✅ **Unified LLM Interface** - Switch between providers with one line  
✅ **Structured Output** - Get validated data with Pydantic models  
✅ **Natural Language SQL** - Query databases conversationally  
✅ **PDF Extraction** - Extract structured data from documents  
✅ **Image Analysis** - Extract data from charts and visualizations  
✅ **Thinking Agent** - Complex reasoning with transparent process  
✅ **Orchestrator** - Multi-agent collaboration for complex tasks  

### What's Next?

1. **Deep Dive Tutorial** (Notebook 2) - Advanced features:
   - Memory management for long conversations
   - Custom tools and function calling
   - Advanced extraction with HTML and text
   - Building production pipelines

2. **Production Patterns** (Notebook 3) - Best practices:
   - Error handling and retries
   - Cost optimization
   - Performance tuning
   - Security considerations

### Try These Challenges:

- Extract data from your own PDFs or images
- Create custom Pydantic schemas for your use case
- Build a data pipeline combining multiple sources
- Compare provider performance on your specific tasks

### Resources:

- Documentation: [defog.ai/docs](https://defog.ai/docs)
- GitHub: [github.com/defog-ai/defog-python](https://github.com/defog-ai/defog-python)
- Support: support@defog.ai

Happy building! 🚀